In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API Credentials
with open('/Users/lbodden/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [4]:
# set our API call parameters 
LOCATION = 'Bronx,NY'
TERM = 'Pizza'

In [5]:
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE


'Data/results_in_progress_NY_pizza.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

7200

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


360

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/360 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/360 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [15]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [16]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [17]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


360

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/360 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [19]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,PBsVApRd0dHVjWgiGZgcRQ,louie-and-ernies-pizza-bronx-2,Louie & Ernie's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/R-p0EL...,False,https://www.yelp.com/biz/louie-and-ernies-pizz...,354,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.838288, 'longitude': -73.828563}","[delivery, pickup]",$,"{'address1': '1300 Crosby Ave', 'address2': ''...",+17188296230,(718) 829-6230,3915.849575
1,b6jOwyX4iaagw8YjXqq1sA,antonios-trattoria-bronx,Antonio's Trattoria,https://s3-media3.fl.yelpcdn.com/bphoto/2JkIBH...,False,https://www.yelp.com/biz/antonios-trattoria-br...,957,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.854131, 'longitude': -73.886601}","[delivery, pickup]",$$,"{'address1': '2370 Belmont Ave', 'address2': N...",+17187336630,(718) 733-6630,1297.605191
2,ZIdfJ9a-tfo1LDhqp9w6TA,franks-pizzeria-bronx,Frank's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/FZAbj8...,False,https://www.yelp.com/biz/franks-pizzeria-bronx...,114,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.8435944773229, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '2823 Middletown Rd', 'address2':...",+17185973333,(718) 597-3333,3129.337817
3,HaXbpA8etMMiqqFNYoltTg,pugsleys-pizza-bronx,Pugsley's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/kKjMII...,False,https://www.yelp.com/biz/pugsleys-pizza-bronx?...,176,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.85881, 'longitude': -73.88475}","[delivery, pickup]",$,"{'address1': '590 E 191st St', 'address2': '',...",+17183650327,(718) 365-0327,1346.294930
4,2tPFIQsuvcS-T0r7NtaKAg,john-and-joes-pizzeria-bronx,John & Joe's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/aOGdSb...,False,https://www.yelp.com/biz/john-and-joes-pizzeri...,190,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.8545949, 'longitude': -73.86578}",[delivery],$,"{'address1': '749 Lydig Ave', 'address2': None...",+17184099776,(718) 409-9776,531.728124


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,x3WAQIO7dPSnHUeUpFTSlQ,napoli-pizzeria-and-restaurant-fresh-meadows,Napoli Pizzeria & Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/50Z6ZS...,False,https://www.yelp.com/biz/napoli-pizzeria-and-r...,154,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.7329651024261, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '16402 69th Ave', 'address2': '',...",+17183801172,(718) 380-1172,14397.883469
996,xluTHx9XUeoAa5cIJdKKOQ,taste-of-italy-pizzeria-jackson-heights-2,Taste of Italy Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/jRhE3o...,False,https://www.yelp.com/biz/taste-of-italy-pizzer...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 40.7486016731124, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '37-72 90th St', 'address2': '', ...",+17185052600,(718) 505-2600,11528.867654
997,zHP6JuFIj6lykn0buQTCeQ,midland-on-kimball-pizzeria-yonkers,Midland On Kimball Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/cDhJ6R...,False,https://www.yelp.com/biz/midland-on-kimball-pi...,9,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.92025032168667, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '558 Kimball Ave', 'address2': ''...",+19144574560,(914) 457-4560,7706.926450
998,v2RkWChvH9sLZYUijLI3zQ,national-restaurant-and-coffee-shop-bronx,National Restaurant & Coffee Shop,https://s3-media3.fl.yelpcdn.com/bphoto/QzDLsE...,False,https://www.yelp.com/biz/national-restaurant-a...,73,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.0,"{'latitude': 40.8729995, 'longitude': -73.8891...","[delivery, pickup]",$,"{'address1': '8 Bedford Park Blvd E', 'address...",+17183655515,(718) 365-5515,2730.406208
999,-KCs1dphF0hNmHfO299S9g,nino-s-beach-port-washington,Nino’s Beach,https://s3-media1.fl.yelpcdn.com/bphoto/CoV7vP...,False,https://www.yelp.com/biz/nino-s-beach-port-was...,50,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 40.83733341591415, 'longitude': -...",[],NaN,"{'address1': '43 Orchard Beach Blvd', 'address...",+15165020441,(516) 502-0441,13003.277351


In [21]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

55

In [22]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)